In [1]:
import argparse
import os
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from PIL import Image
import models
import vrProjector

In [2]:
# Default input size

height = 512
width = 512
channels = 3
batch_size = 1
input_node = None
net = None
image_path = "room.jpg"
model_data_path = "models/NYU_ResNet-UpProj.npy"

In [3]:
# Read image
img = Image.open(image_path)
img = img.resize([width,height], Image.ANTIALIAS)
img = np.array(img).astype('float32')
img = np.expand_dims(np.asarray(img), axis = 0)

In [4]:
# Create a placeholder for the input image
input_node = tf.placeholder(tf.float32, shape=(None, height, width, channels))

# Construct the network
net = models.ResNet50UpProj({'data': input_node}, batch_size, 1, False)

In [5]:
sess = tf.Session()
# Load the converted parameters
print('Loading the model')

# Use to load from ckpt file
#saver = tf.train.Saver()     
#saver.restore(sess, model_data_path)

# Use to load from npy file
net.load(model_data_path, sess)

Loading the model


In [6]:
def myPredictFace(faceArray,faceName):
    img = Image.fromarray(faceArray)
    img = img.resize([width,height], Image.ANTIALIAS)
    plt.imshow(img,interpolation="bicubic")
    img = np.array(img).astype('float32')
    img = np.expand_dims(np.asarray(img), axis = 0)

    pred = sess.run(net.get_output(), feed_dict={input_node: img})
    # Plot result
    fig = plt.figure()
    ii = plt.imshow(pred[0,:,:,0], interpolation='nearest' )
    fig.colorbar(ii)
    plt.show()
    cmap = plt.cm.viridis
    plt.imsave(faceName +'.png', pred[0,:,:,0],  cmap=cmap, vmin = 0, vmax = 5)
    res = pred[0].flatten()
    res = np.repeat(res,4)
    res.astype('float32').tofile(faceName + '.bytes')

In [8]:
eq = vrProjector.EquirectangularProjection()
eq.loadImage("skybox/Courtyard.jpg")
cb = vrProjector.CubemapProjection()
cb.initImages(512,512)
cb.reprojectToThis(eq)

In [ ]:
myPredictFace(cb.front,'nz')

In [ ]:
myPredictFace(cb.front,'nz')
myPredictFace(cb.back,'pz')
myPredictFace(cb.right,'px')
myPredictFace(cb.left,'nx')
myPredictFace(cb.top,'py')
myPredictFace(cb.bottom,'ny')

In [ ]:
cb2 = vrProjector.CubemapProjection()
cb2.loadImages("nz.png", "px.png", "pz.png", "nx.png", "py.png", "ny.png")
eq2 = vrProjector.EquirectangularProjection()
eq2.initImage(2048,1024)
eq2.reprojectToThis(cb2)
eq2.saveImage("Panorama.png")